<a href="https://colab.research.google.com/github/hargurjeet/Maersk/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [2]:
## Python Imports
import pandas as pd
import numpy as np
from git import Repo
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Clone the GitHub repository
repo_url = 'https://github.com/hargurjeet/Maersk'
repo_dir = 'Maersk'
Repo.clone_from(repo_url, repo_dir)

<git.repo.base.Repo '/content/Maersk/.git'>

In [24]:
import pandas as pd

file_path = "/content/Maersk/vessel_data.csv"
date_cols = ['arrival_date', 'departure_date', 'arrival_time', 'departure_time', 'prev_arrival_time', 'prev_departure_time', 'next_arrival_time']

dtypes = {
    'imo': 'object',
}

vessel_data = pd.read_csv(file_path, parse_dates=date_cols, dtype=dtypes,)
pd.set_option('display.max_columns', None)
vessel_data.head(2)


,Unnamed: 0,call_id,arrival_date,departure_date,arrival_time,departure_time,imo,vessel_name_ais,vessel_type_ais,Country,CountryCode,Port,Terminal,Duration,Service_name,length,prev_departure_time,vesteu,prev_port,prev_arrival_time,prev_duration,prev_leg_duration,prev_leg_distance_nm,prev_leg_stationary_hours,prev_call_id,next_port,next_arrival_time,next_duration,next_leg_duration,next_leg_distance_nm,next_leg_stationary_hours,next_call_id,total_duration_shipment,total_duration_shipment_days,total_distance_shipment
0,0,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,2022-01-05,2022-01-07,2022-01-05 23:00:19+00:00,2022-01-07 21:04:26+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,Xiamen,Xiamen Songyu Container Terminal,46.068611,ML - TPX,249.12,2021-12-14 07:00:05+00:00,4496,Haiphong,2021-12-13 17:02:55+00:00,13.952778,544.003889,1598.620103,28,705238686bb73f1947352631d015ad9485d78a9c,Los Angeles,2022-02-10 12:01:14+00:00,207.020556,806.946667,7407.903024,201.0,c77797c9b9822d7a799e6f7664f7d6be5f73f270,1846.992500,76.958021,9006.523126
1,1,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14+00:00,2022-02-19 03:02:28+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,Los Angeles,Apmt Pier 400,207.020556,ML - TPX,249.12,2022-01-07 21:04:26+00:00,4496,Xiamen,2022-01-05 23:00:19+00:00,46.068611,806.946667,7407.903024,201,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,Qingdao,2022-03-08 22:27:41+00:00,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,1819.405278,75.808553,13374.983265


In [33]:
# Remove timezone information from the date-time strings
vessel_data['arrival_time'] = vessel_data['arrival_time'].str[:-6]
vessel_data['departure_time'] = vessel_data['departure_time'].str[:-6]
vessel_data['prev_arrival_time'] = vessel_data['prev_arrival_time'].str[:-6]
vessel_data['prev_departure_time'] = vessel_data['prev_departure_time'].str[:-6]
vessel_data['next_arrival_time'] = vessel_data['next_arrival_time'].str[:-6]


# Convert 'arrival_time' to datetime with UTC timezone
vessel_data['arrival_time'] = pd.to_datetime(vessel_data['arrival_time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
vessel_data['departure_time'] = pd.to_datetime(vessel_data['departure_time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
vessel_data['prev_arrival_time'] = pd.to_datetime(vessel_data['prev_arrival_time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
vessel_data['prev_departure_time'] = pd.to_datetime(vessel_data['prev_departure_time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
vessel_data['next_arrival_time'] = pd.to_datetime(vessel_data['next_arrival_time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

In [34]:
vessel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494042 entries, 0 to 494041
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Unnamed: 0                    494042 non-null  int64         
 1   call_id                       494042 non-null  object        
 2   arrival_date                  494042 non-null  datetime64[ns]
 3   departure_date                494042 non-null  datetime64[ns]
 4   arrival_time                  493604 non-null  datetime64[ns]
 5   departure_time                493614 non-null  datetime64[ns]
 6   imo                           494042 non-null  object        
 7   vessel_name_ais               494042 non-null  object        
 8   vessel_type_ais               494039 non-null  object        
 9   Country                       494042 non-null  object        
 10  CountryCode                   493669 non-null  object        
 11  Port         

In [31]:
vessel_data['arrival_time']

0        2022-01-05 23:00:19
1        2022-02-10 12:01:14
2        2022-03-08 22:27:41
3        2022-06-18 04:02:02
4        2022-06-23 03:00:26
                 ...        
494037   2023-12-09 03:01:13
494038   2023-12-15 22:00:06
494039   2023-12-18 17:00:20
494040   2023-12-21 17:00:36
494041   2023-12-25 08:00:14
Name: arrival_time, Length: 494042, dtype: datetime64[ns]